<a href="https://colab.research.google.com/github/RuzbehAmrolia/covid_vaccine_tweets/blob/main/Naive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Embedding,Bidirectional,LSTM,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import preprocessor as p
from sklearn.naive_bayes import MultinomialNB
import re
import os
from textblob import TextBlob

In [5]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
from keras.models import model_from_json

In [6]:
wn = nltk.WordNetLemmatizer()
lemmatizer = WordNetLemmatizer()

In [7]:
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [8]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt')
stopwords = set(STOPWORDS)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
from google.colab import files
uploaded = files.upload()

Saving vaccination_all_tweets.csv to vaccination_all_tweets.csv


In [13]:
df=pd.read_csv('vaccination_all_tweets.csv')
df.drop_duplicates(subset = "text", keep = "first",inplace = True, ignore_index = True) #Removing duplicates
df.head()

,user_location,date,text,hashtags,retweets,favorites,is_retweet
0,"La Crescenta-Montrose, CA",2020-12-20 6:06,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],0,0,False
1,"San Francisco, CA",2020-12-13 16:27,While the world has been on the wrong side of ...,NaN,1,1,False
2,Your Bed,2020-12-12 20:33,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",0,0,False
3,"Vancouver, BC - Canada",2020-12-12 20:23,"Facts are immutable, Senator, even when you're...",NaN,446,2129,False
4,NaN,2020-12-12 20:17,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",0,0,False


In [14]:
tweet_list = []
for x in df["text"]:
    tweet_list.append(p.clean(x)) #Cleaning tweets (Removes URLs, Hashtags, Mentions, Reserved Words (RT,FAV) Emojis, Smileys) and appending to a list
    
tweet_list = [re.sub('[^a-zA-Z]', ' ',i) for i in tweet_list] #Removing punctuatuons

print("----Before----")
print(df["text"][3])

print("----After----")
print(tweet_list[3])

----Before----
Facts are immutable, Senator, even when you're not ethically sturdy enough to acknowledge them. (1) You were born i… https://t.co/jqgV18kch4
----After----
Facts are immutable  Senator  even when you re not ethically sturdy enough to acknowledge them      You were born i


In [15]:
clean_tweets = []
for tweet in tweet_list: 
  for word in set(stopwords):
    token = " "+word+" "
    tweet = tweet.replace(token, " ")
  clean_tweets.append(tweet)


In [16]:
lemmenn = []
for word in clean_tweets:
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(word)) 
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
        # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    lemmatized_sentence = " ".join(lemmatized_sentence)
    lemmenn.append(lemmatized_sentence)

i=2
print(clean_tweets[i])
print(lemmenn[i])


Russian vaccine created last    years
Russian vaccine create last year


In [17]:
df["Processed"] = lemmenn
df.Processed = df.Processed.apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df = df.dropna()

tokenized_tweet = df['Processed'].apply(lambda x: x.split())
tokenized_tweet.head()

0     [Same, folk, daikon, paste, treat, cytokine, s...
2                [Russian, vaccine, create, last, year]
6     [claim, fame, success, patriotic, competition,...
9                         [States, will, start, Monday]
10    [while, death, close, mark, million, people, w...
Name: Processed, dtype: object

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(ngram_range=(1,1), max_features=150).fit(df["Processed"]) 
trans = vector.transform(df["Processed"])  # Transforming the vectorizer
df_vect=pd.DataFrame(trans.toarray(), columns=vector.get_feature_names()) # Converting to DataFrame

df_vect.head(10)

,administer,after,already,announce,anyone,approval,approve,arrive,available,back,batch,become,begin,best,bharat,biotech,business,call,canada,case,china,chinese,clinical,come,company,coronavirus,country,covaxin,covid,data,death,develop,dose,drug,effect,effective,efficacy,embarrassing,emergency,even,...,sore,sputnik,start,state,stay,still,study,supply,take,test,thank,thanks,think,third,this,time,today,tomorrow,toronto,trial,update,usual,vaccinate,vaccinated,vaccination,vaccine,vaccines,variant,wait,want,week,well,what,while,will,with,work,world,year,yesterday
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.233184,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.55638,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.779022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.626996,0.0,0.0,0.0,0.00000,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.467279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.444648,0.0,0.0,0.0,0.0,0.471273,0.000000,0.0,0.0,0.0,0.00000,0.0
5,0.0,0.0,0.0,0.572502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.425256,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.389934,0.0,0.0,0.0,0.00000,0.0
6,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0
7,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.362978,0.0,0.0,0.0,0.0,0.0,0.426953,0.0,0.0,0.0,0.0,0.446571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0
8,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.00000,0.0
9,0.0,0.0,0.0,0.

In [19]:
pol = []
for tweet in df["Processed"]:
    blob = TextBlob(tweet)
    pol.append(blob.sentiment.polarity)
df['Polarity']=pol


In [20]:
positive ,negative, neutral = 0,0,0
for polar in df["Polarity"]:
    if polar > 0:
        positive += 1
    elif polar < 0:
        negative += 1
    else:
        neutral += 1
        
pol_list = []

for x in pol:
    if x>0:
        pol_list.append("Positive")
    elif x<0:
        pol_list.append("Negative")
    else:
        pol_list.append("Neutral")
        
df["Sentiment"] = pol_list

print(df.Sentiment.value_counts())


Neutral     23465
Positive    14750
Negative     4242
Name: Sentiment, dtype: int64


In [21]:
label = list(df["Sentiment"].unique())
labl = []
labl.append(label[0])
labl.append(label[2])
labl.append(label[1])
values = df["Sentiment"].value_counts()
label = labl
label

sentences = list(df.Processed)
labels = df.Sentiment.tolist()

In [22]:
#Converting any float values in the processed tweets into string

ctr2 = 0 
for x in sentences:
    if type(x) == float:
        ctr2 += 1
        sentences[sentences.index(x)] = "nan"


In [23]:
#--------------------------Naive Bayes-----------------------------#
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier


In [24]:
#get percentage of positive tweets
ptweets = df[df.Sentiment == "Positive"]
ptweets = ptweets["Processed"]

features_positive = [(f, 
           'Positive') for f in ptweets]

#get percentage of negative tweets
ntweets = df[df.Sentiment == "Negative"]
ntweets = ntweets["Processed"]
features_negative = [(f, 
           'Negative') for f in ntweets]

#get percentage of neutral tweets
ntweets = df[df.Sentiment == "Neutral"]
ntweets = ntweets["Processed"]
features_neutral = [(f, 
           'Neutral') for f in ntweets]

In [25]:
# Split the data into train and test (80/20)
threshold_factor = 0.7
threshold_pos = int(threshold_factor * len(features_positive))
threshold_neg = int(threshold_factor * len(features_negative))
threshold_neut = int(threshold_factor * len(features_neutral))

In [26]:
#create training set and testing set
features_train = features_positive[:threshold_pos] + features_negative[:threshold_neg] #+ features_neutral[:threshold_neut]
features_test = features_positive[threshold_pos:] + features_negative[threshold_neg:]  #+ features_neutral[threshold_neut:]
print("\nNumber of training datapoints:", len(features_train))
print("Number of test datapoints:", len(features_test))


Number of training datapoints: 13294
Number of test datapoints: 5698


In [27]:
all_words = set(word.lower() for passage in features_train for word in word_tokenize(passage[0]))
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in features_train]

In [28]:
test_words = set(word.lower() for passage in features_test for word in word_tokenize(passage[0]))
t1 = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1]) for x in features_test]

In [29]:
# Train a Naive Bayes classifier
classifier = NaiveBayesClassifier.train(t)
print("\nAccuracy of the classifier:", nltk.classify.util.accuracy(classifier, t1))


Accuracy of the classifier: 0.885047385047385


In [ ]:
print("\nTop 20 most informative words:")
for item in classifier.most_informative_features()[:50]:
    print(item[0])

In [41]:
# Sample input tweets gathered by randomly selecting tweets on the #covidvaccine hashtag
input_tweets = [
    "For months, I've been criticizing the CDC for being too cautious with what vaccinated people can do. Now, the CDC has done an about-face that's shockingly abrupt: it's confusing & could actually disincentivize vaccines. We need an intermediate step: Thread.", 
    "I hope vaccine will help to fight covid 19 ...after taking two vaccines, I will not have a Corona..",
    "You cannot catch coronavirus from the #COVIDVaccine. There’s a lot of misinformation out there, so make sure you’re getting it from a trusted source. For more information on the COVID vaccine, visit http://nhs.uk/covidvaccine.", 
    "My anti-mask brother, wife & sons who infected my mom with Covid at my fathers funeral, will now not help her get a vaccine. Mom (82) is sneaking calls to my daughter for help to get on the list. They also tried to infect my husband at the funeral-he has severe heart problems",
    "Our lying politicians.....this webpage has not been updated since 17th June 2020....where is the mention of the 2 new variants? Where...",
    "Consignment of Covishield vaccine is being loaded onto the aircrafts for distribution all over the country now",
    "2nd COVID vaccine down. Expecting some not fun side effects, but totally worth it",
    "I want to ask that why many are feeling tired , having fever and weakness after taking 1st dose or 2nd dose of VACCINE",
    "If the AstraZeneca vaccine is not safe for the 1st dose, what makes you believe it is safe for the 2nd dose?"
   ]

In [42]:
print("Predictions:")
for tweet in input_tweets:
    print("Tweet: ", tweet)
    probdist = classifier.prob_classify(extract_features(tweet.split()))
    pred_sentiment = probdist.max()
    print("Predicted sentiment:", pred_sentiment)
    print("Probability:", round(probdist.prob(pred_sentiment), 2))

Predictions:
Tweet:  For months, I've been criticizing the CDC for being too cautious with what vaccinated people can do. Now, the CDC has done an about-face that's shockingly abrupt: it's confusing & could actually disincentivize vaccines. We need an intermediate step: Thread.
Predicted sentiment: Negative
Probability: 0.99
Tweet:  I hope vaccine will help to fight covid 19 ...after taking two vaccines, I will not have a Corona..
Predicted sentiment: Positive
Probability: 0.69
Tweet:  You cannot catch coronavirus from the #COVIDVaccine. There’s a lot of misinformation out there, so make sure you’re getting it from a trusted source. For more information on the COVID vaccine, visit http://nhs.uk/covidvaccine.
Predicted sentiment: Positive
Probability: 0.99
Tweet:  My anti-mask brother, wife & sons who infected my mom with Covid at my fathers funeral, will now not help her get a vaccine. Mom (82) is sneaking calls to my daughter for help to get on the list. They also tried to infect my h

In [60]:
#testing
custom_tweet = "There is a shortage of vaccines in India and the government has closed the vaccination drive for 18-44 years."
classifier.classify(dict([token, True] 
                         for token in extract_features(word_tokenize(custom_tweet))))

'Negative'

In [61]:
#testing
custom_tweet = "BIG : Bharat Biotech Nasal Vaccines Awaiting Regulatory Approval ; Will be First Nasal Drops in the World that will Stop Infection & Transmission : MD"
classifier.classify(dict([token, True] 
                         for token in extract_features(word_tokenize(custom_tweet))))

'Positive'

In [43]:
import pickle
with open('/content/gdrive/MyDrive/Models/Nbayes.pickle','wb') as f:
    pickle.dump(classifier,f)
    print("Model saved to drive")

Model saved to drive


In [52]:
classifier_f = open("/content/gdrive/MyDrive/Models/Nbayes.pickle", "rb")
classifier = pickle.load(classifier_f)
print("Model loaded from drive")
classifier_f.close()

Model loaded from drive


In [57]:
clf = print(classifier.labels())

['Positive', 'Negative']


In [58]:
print('Naive Bayes Confusion Matrix' , nltk.ConfusionMatrix(labels, tests))
print('Naive Bayes Classification Report', classification_report(labels, tests))


NameError: ignored